In [13]:
import json
import requests  
import os
import pandas as pd
import time

In [14]:
# Get porfolio 
def get_portfolio(data):
    # Retrive date, fund name and portfolio composition
    fecha = data['data'][0]['dataXML']['Cabecera']['FechaReporte']
    nombre_fondo = data['data'][0]['dataXML']['Cabecera']['FondoNombre']
    nombre_archivo = nombre_fondo+'_' + fecha +'.json'

    # Create a dictionary for each fund each day
    portfolio = data['data'][0]['dataXML']
    cartera = {}
    cartera['Api']= fund
    cartera['Fecha'] = fecha 
    cartera['Nombre_Fondo'] = nombre_fondo
    cartera['Portfolio'] = portfolio
    
    return cartera , fecha , nombre_fondo

In [15]:
# Get last day 
with open('last_day.json', 'r') as last_day:
    ultima_fecha = json.load(last_day)
last_day.close()

# Get next fund to download
with open('start_fund.json', 'r') as infile:
    fund = json.load(infile)
    
#Get next portfolio    
api="https://api.cafci.org.ar/interfaz/semanal/resumen/cartera/"
website = api+str(fund)
# infile.close()

r = requests.get(website)
status_code = r.status_code
data = r.json()

In [16]:
while (status_code == 200 and data["success"]):
    
    cartera , fecha_fondo , nombre_fondo = get_portfolio(data)
    print(fund , fecha_fondo)
    
    # Check if day file already exists
    with open('days_list.json','r') as days_file:
        past_days = json.load(days_file) 
    days_file.close()
    
    if fecha_fondo not in past_days:
        Exist = False        
        past_days.append(fecha_fondo)
        with open('days_list.json', 'w') as outfile:
            json.dump(past_days, outfile)  
    else:
        Exist = True
        
    # Check where to save new data
    file_name = fecha_fondo + '.json'
   
    if fecha_fondo == ultima_fecha: 
        with open(file_name,'r') as old_data:
            fondos_histo = json.load(old_data)
        old_data.close()
            
#         if nombre_fondo not in fondos_histo[fecha_fondo].keys():
        fondos_histo[fecha_fondo][nombre_fondo] = {}
        fondos_histo[fecha_fondo][nombre_fondo] = cartera
        
            # Save histo data in json file
        with open(file_name, 'w') as outfile:
            json.dump(fondos_histo, outfile)
        outfile.close()
#         976375
    elif fecha_fondo != ultima_fecha:
        if Exist == False:       
            # else create dictionary
            fondos_new = {}
            fondos_new[fecha_fondo] = {}
            fondos_new[fecha_fondo][nombre_fondo] = cartera
            # Save histo data in new json file
            with open(file_name, 'w') as first_outfile:
                json.dump(fondos_new, first_outfile)
            first_outfile.close()
            
        # file already exits but day is already on the list 
        elif Exist == True: 
            with open(file_name,'r') as old_data:
                fondos_histo = json.load(old_data)
            old_data.close()

            # Save data 
#             if nombre_fondo not in fondos_histo[fecha_fondo].keys():
            fondos_histo[fecha_fondo][nombre_fondo] = {}
            fondos_histo[fecha_fondo][nombre_fondo] = cartera
#                 print(fondos_histo[fecha_fondo].keys())

            # Save histo data in json file
            with open(file_name, 'w') as outfile:
                json.dump(fondos_histo, outfile)
            outfile.close()
          
    # Save last_day
    ultima_fecha = fecha_fondo
    with open('last_day.json', 'w') as last_day:
        json.dump(ultima_fecha,last_day)
    last_day.close()

    # Save next fund number
    fund= fund + 1
    with open('start_fund.json', 'w') as start_fund:
        json.dump(fund,start_fund)
#1260000
    # Reach api fund
    api="https://api.cafci.org.ar/interfaz/semanal/resumen/cartera/"
    website = api+str(fund)
    r = requests.get(website)
    status_code = r.status_code
    data = r.json()

KeyError: 'success'